# Importing modules

In [ ]:
import selenium
from selenium import webdriver
import time
from urllib.request import unquote
from selenium.webdriver.common.keys import Keys
import urllib.request
from pdf2image import convert_from_path
from pdf2image import convert_from_path
import pymongo
import base64
import bson
from bson.binary import Binary
import gridfs
import os
from selenium.webdriver.chrome.options import Options
chop = Options()

# Defining functions

This function will allow us to store all the links of the financial documents in a list

In [ ]:
def collect_links():
    options = Options()
    options.add_argument("start-maximized")
    driver=webdriver.Chrome(r"C:\Users\HP\Desktop\Data Science\Projet\chromedriver.exe")
    driver.set_page_load_timeout(3000)
    driver.get("http://www.bvmt.com.tn/fr/entreprises/list")
    time.sleep(5)
    lst = list()
    select_box = driver.find_element_by_xpath('//*[@id="ext-gen1009"]') 
    options = [x for x in select_box.find_elements_by_tag_name('option')]
    for element in options:
        if (element.get_attribute("uri") != None ):
                lst.append({"nom" : element.get_attribute("value"),"lien":"http://www.bvmt.com.tn"+element.get_attribute("uri"),'lienPDF_1':"",'lienPDF_2':"",'lienPDF_3':""})
    driver.quit()
    return lst

This function will allow us to download the financial documents of 3 years of each company and store the file's names in a list that will be used later

In [ ]:
def collect_financial_Documents(lst):
    options = Options()
    options.add_argument("start-maximized")
    driver=webdriver.Chrome(r"C:\Users\HP\Desktop\Data Science\Projet\chromedriver.exe")
    driver.set_page_load_timeout(3000)

    
      
    for elem in lst : 
        #acceder a la page de la societé
        driver.get(elem["lien"])
        #acceder a la page des etats
        driver.find_element_by_xpath('//*[@id="my-menu"]/li[14]/a').click()
        list_ann= list()
        select = driver.find_element_by_xpath('//*[@id="ext-gen1009"]')
        content = [x for x in select.find_elements_by_tag_name('div')]
        case=0
        for b in content :
            if(b.get_attribute('onclick') != None) :
                if("annuels-consolid" in b.get_attribute('onclick') ) or ("annuels-3" in b.get_attribute('onclick')) :
                    a=b.get_attribute('onclick')
                    ch=a[len(a)-5:len(a)-1]
                    if(case==0):
                        premier_slash=b.get_attribute('onclick').find("/")
                        list_ann.append("http://www.bvmt.com.tn"+b.get_attribute('onclick')[premier_slash:-1])
                        case=case+ 1
                    elif(ch not in list_ann[-1]):
                        premier_slash=b.get_attribute('onclick').find("/")
                        list_ann.append("http://www.bvmt.com.tn"+b.get_attribute('onclick')[premier_slash:-1])
                        case=case+ 1
        ann=1
        for c in list_ann :
            if(ann<=3):
                driver.get(c)
                lien = driver.find_element_by_xpath('//*[@id="bt-pdflong"]/a')
                annee='lienPDF_'+str(ann)
                elem[annee] = lien.get_attribute('href')
                filename = unquote(elem[annee]).split('/')[-1].replace(' ', '_')
                names_list.append(filename)

                urllib.request.urlretrieve(lien.get_attribute('href'), filename)
                ann=ann+1
    driver.quit()

    return names_list

This function will allow us to upload the files on MongoDB

In [ ]:
def save_in_MongoDB(path):
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['ProjetFinance'] 
    fs = gridfs.GridFS(db)
    # Note, open with the "rb" flag for "read bytes"
    with open(path, "rb") as f:
        encoded_string = base64.b64encode(f.read())
    with fs.new_file(
        chunkSize=800000,
        filename=path) as fp:
        fp.write(encoded_string)


This function will allow us to upload the files on MongoDB

In [ ]:
def read_from_MongoDB(filename):
    # Usual setup
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['financial'] 
    fs = gridfs.GridFS(db)
    # Standard query to Mongo
    data = fs.find_one(filter=dict(filename=filename))
    with open(filename, "wb") as f:
        f.write(base64.b64decode(data.read()))
        


This function will allow us to convert pdf into image

In [ ]:
def pdf2image(pdfName):
    pages = convert_from_path(pdfName, 500,poppler_path=r'C:\Program Files\poppler-0.68.0\bin')
    newpath = r'C:\Users\HP\Desktop\Notebooks\image\''+pdfName
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for i, image in enumerate(pages):
        fname = newpath+'\image'+str(i)+'.png'
        image.save(fname, "PNG")

    

# Collecting Data

### Web Scraping

Step 1 : Collecting all the links needed

In [ ]:
links=[]
links=collect_links()
print("The Number of companies are : "+str(len(links)))

The Number of companies are : 80


Step 2 : Downloading the documents

In [ ]:
names_list=[]
names_list=collect_financial_Documents(links)

In [ ]:
print("The Number of documents : "+str(len(names_list)))

The Number of documents : 231


In [ ]:
names_list

['ab-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ab-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ab-etats-financiers-annuels-31-12-2017.pdf',
 'adwya-etats-financiers-annuels-31-12-2019.pdf',
 'adwya-etats-financiers-annuels-31-12-2018.pdf',
 'adwya-etats-financiers-annuels-31-12-2016.pdf',
 'aetec-etats-financiers-annuels-31-12-2019.pdf',
 'aetec-etats-financiers-annuels-31-12-2018.pdf',
 'aetec-etats-financiers-annuels-31-12-2017.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'alkim-etats-financiers-annuels-31-12-2018.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'ams-etats-financiers-annuels-31-12-2017.pdf',
 'ams-etats-financiers-annuels-consolides-31-12-2015.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'artes-etats-financier

Step 3 : Storing the documents on MongoDB

In [ ]:
for filename in names_list :
    save_in_MongoDB(filename)

Step 4 : Convert pdf to image

In [ ]:
for filename in names_list :
    pdf2image(filename)

In [ ]:
a=['ab-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ab-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ab-etats-financiers-annuels-31-12-2017.pdf',
 'adwya-etats-financiers-annuels-31-12-2019.pdf',
 'adwya-etats-financiers-annuels-31-12-2018.pdf',
 'adwya-etats-financiers-annuels-31-12-2016.pdf',
 'aetec-etats-financiers-annuels-31-12-2019.pdf',
 'aetec-etats-financiers-annuels-31-12-2018.pdf',
 'aetec-etats-financiers-annuels-31-12-2017.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'alkim-etats-financiers-annuels-31-12-2018.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ams-etats-financiers-annuels-31-12-2017.pdf',
 'ams-etats-financiers-annuels-consolides-31-12-2015.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'artes-etats-financiers-annuels-31-12-2018.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'assad-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'assad-etats-financiers-annuels-31-12-2018.pdf',
 'assad-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ast-etats-financiers-annuels-31-12-2019.pdf',
 'ast-etats-financiers-annuels-31-12-2018.pdf',
 'ast-etats-financiers-annuels-31-12-2016.pdf',
 'atb-etats-financiers-annuels-31-12-2019.pdf',
 'atb-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'atb-etats-financiers-annuels-31-12-2018.pdf',
 'atl-etats-financiers-annuels-consolides-31-12-2020.pdf',
 'atl-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'atl-etats-financiers-annuels-consolides-31-12-2019_0.pdf',
 'tjari-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'tjari-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tjari-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'tjl-etats-financiers-annuels-31-12-2019.pdf',
 'tjl-etats-financiers-annuels-31-12-2017.pdf',
 'tjl-etats-financiers-annuels-31-12-2016.pdf',
 'maghrebia-etats-financiers-annuels-31-12-2019.pdf',
 'maghrebia-etats-financiers-annuels-consolides-31-12-2018_0.pdf',
 'maghrebia-etats-financiers-annuels-31-12-2017.pdf',
 'bl-etats-financiers-annuels-31-12-2020.pdf',
 'bl-etats-financiers-annuels-31-12-2018.pdf',
 'bl-etats-financiers-annuels-31-12-2017_1.pdf',
 'bh-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'bh-etats-financiers-annuels-consolides-31-12-2019_1.pdf',
 'bh-etats-financiers-annuels-31-12-2018_0.pdf',
 'bh-l-etats-financiers-annuels-31-12-2019.pdf',
 'ml-etats-financiers-annuels-31-12-2018.pdf',
 'ml-etats-financiers-annuels-31-12-2017.pdf',
 'biat-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'biat-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'biat-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'bna-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'bna-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'bna-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'bt-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'bt-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'bt-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'bte-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'bte-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'bte-etats-financiers-annuels-31-12-2018.pdf',
 'bh-assurance-etats-financiers-annuels-31-12-2020.pdf',
 'bh-assurance-etats-financiers-annuels-31-12-2019.pdf',
 'salim-etats-financiers-annuels-31-12-2018.pdf',
 'cc-etats-financiers-annuels-31-12-2020.pdf',
 'cc-etats-financiers-annuels-31-12-2019.pdf',
 'cc-etats-financiers-annuels-31-12-2018.pdf',
 'cell-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'cell-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'cell-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'creal-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'creal-etats-financiers-annuels-31-12-2018_0.pdf',
 'creal-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'cil-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'cil-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'cil-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'scb-etats-financiers-annuels-31-12-2019.pdf',
 'scb-etats-financiers-annuels-31-12-2018.pdf',
 'scb-etats-financiers-annuels-31-12-2017.pdf',
 'city-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'city-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'city-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'dh-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'dh-etats-financiers-annuels-31-12-2018.pdf',
 'dh-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'lstr-etats-financiers-annuels-31-12-2019.pdf',
 'lstr-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'lstr-etats-financiers-annuels-31-12-2018.pdf',
 'nakl-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'nakl-etats-financiers-annuels-consolides-31-12-2019_0.pdf',
 'sokna-etats-financiers-annuels-31-12-2020.pdf',
 'sokna-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sokna-etats-financiers-annuels-31-12-2018.pdf',
 'ecycl-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ecycl-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'gif-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'gif-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'gif-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'hl-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'hl-etats-financiers-annuels-31-12-2018.pdf',
 'hl-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'icf-etats-financiers-annuels-31-12-2020.pdf',
 'icf-etats-financiers-annuels-31-12-2019.pdf',
 'icf-etats-financiers-annuels-31-12-2018.pdf',
 'lndor-etats-financiers-annuels-31-12-2019.pdf',
 'lndor-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'lndor-etats-financiers-annuels-31-12-2018.pdf',
 'mag-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'mag-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'mag-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'mip-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'mip-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'mip-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'mnp-etats-financiers-annuels-31-12-2019.pdf',
 'mnp-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'mnp-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'mpbs-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'mpbs-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'mpbs-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'nbl-etats-financiers-annuels-31-12-2019.pdf',
 'nbl-etats-financiers-annuels-31-12-2018.pdf',
 'nbl-etats-financiers-annuels-31-12-2017.pdf',
 'oth-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'oth-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'office-plast-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'office-plast-etats-financiers-annuels-31-12-2018.pdf',
 'office-plast-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'pltu-etats-financiers-annuels-31-12-2020.pdf',
 'pltu-etats-financiers-annuels-31-12-2019.pdf',
 'pltu-etats-financiers-annuels-31-12-2018.pdf',
 'pgh-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'pgh-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'sah-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sah-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'sam-etats-financiers-annuels-31-12-2019.pdf',
 'sam-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sam-etats-financiers-annuels-31-12-2018.pdf',
 'smd-etats-financiers-annuels-31-12-2019.pdf',
 'smd-etats-financiers-annuels-31-12-2018.pdf',
 'smd-etats-financiers-annuels-31-12-2017.pdf',
 'servi-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'servi-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'servi-etats-financiers-annuels-31-12-2017.pdf',
 'sfbt-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sfbt-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'sfbt-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'siame-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'siame-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'simpa-etats-financiers-annuels-31-12-2019.pdf',
 'simpa-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'simpa-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'sipha-etats-financiers-annuels-31-12-2018.pdf',
 'sipha-etats-financiers-annuels-31-12-2017.pdf',
 'sipha-etats-financiers-annuels-31-12-2016.pdf',
 'sits-etats-financiers-annuels-consolides-31-12-2020.pdf',
 'sits-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sits-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'somoc-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'somoc-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'somoc-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'sopat-etats-financiers-annuels-31-12-2019.pdf',
 'sopat-etats-financiers-annuels-31-12-2018.pdf',
 'sopat-etats-financiers-annuels-31-12-2017.pdf',
 'sotem-etats-financiers-annuels-31-12-2019.pdf',
 'sotem-etats-financiers-annuels-31-12-2018.pdf',
 'sotem-etats-financiers-annuels-31-12-2017.pdf',
 'sotet-etats-financiers-annuels-31-12-2018.pdf',
 'sotet-etats-financiers-annuels-31-12-2016.pdf',
 'sotet-etats-financiers-annuels-31-12-2015.pdf',
 'stpap-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'stpap-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'stpap-etats-financiers-annuels-31-12-2017.pdf',
 'stpil-etats-financiers-annuels-31-12-2019.pdf',
 'stpil-etats-financiers-annuels-31-12-2018.pdf',
 'stpil-etats-financiers-annuels-31-12-2017.pdf',
 'mgr-etats-financiers-annuels-31-12-2019_1.pdf',
 'mgr-etats-financiers-annuels-31-12-2019.pdf',
 'mgr-etats-financiers-annuels-31-12-2018.pdf',
 'sotuv-etats-financiers-annuels-31-12-2019.pdf',
 'sotuv-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'sotuv-etats-financiers-annuels-31-12-2018.pdf',
 'spdit-etats-financiers-annuels-31-12-2020.pdf',
 'spdit-etats-financiers-annuels-31-12-2019.pdf',
 'spdit-etats-financiers-annuels-31-12-2018.pdf',
 'star-etats-financiers-annuels-31-12-2019.pdf',
 'star-etats-financiers-annuels-31-12-2018.pdf',
 'star-etats-financiers-annuels-31-12-2017.pdf',
 'stb-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'stb-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'stb-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'steq-etats-financiers-annuels-31-12-2019.pdf',
 'steq-etats-financiers-annuels-31-12-2018.pdf',
 'steq-etats-financiers-annuels-31-12-2017.pdf',
 'stip-etats-financiers-annuels-consolides-31-12-2020.pdf',
 'stip-etats-financiers-annuels-31-12-2018.pdf',
 'stip-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tgh-etats-financiers-annuels-30-06-2019.pdf',
 'tgh-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tgh-etats-financiers-annuels-31-12-2017.pdf',
 'tlnet-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'tlnet-etats-financiers-annuels-31-12-2018.pdf',
 'tlnet-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tpr-etats-financiers-annuels-31-12-2018.pdf',
 'tpr-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tpr-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'tinv-etats-financiers-annuels-31-12-2019.pdf',
 'tinv-etats-financiers-annuels-31-12-2018.pdf',
 'tinv-etats-financiers-annuels-31-12-2017.pdf',
 'tre-etats-financiers-annuels-31-12-2019.pdf',
 'tre-etats-financiers-annuels-31-12-2018.pdf',
 'tre-etats-financiers-annuels-31-12-2017.pdf',
 'tair-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'tair-etats-financiers-annuels-consolides-31-12-2016.pdf',
 'tair-etats-financiers-annuels-consolides-31-12-2015_1.pdf',
 'tls-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'tls-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'tls-etats-financiers-annuels-31-12-2017.pdf',
 'uadh-etats-financiers-annuels-31-12-2017.pdf',
 'uadh-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'uadh-etats-financiers-annuels-consolides-31-12-2016.pdf',
 'ubci-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ubci-etats-financiers-annuels-31-12-2018.pdf',
 'ubci-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'uib-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'uib-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'uib-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'unimed-etats-financiers-annuels-31-12-2019.pdf',
 'unimed-etats-financiers-annuels-31-12-2018.pdf',
 'unimed-etats-financiers-annuels-31-12-2017.pdf',
 'wifack-international-bank-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'wifack-international-bank-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'wifack-international-bank-etats-financiers-annuels-consolides-31-12-2017.pdf']



In [ ]:
a

['ab-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'ab-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ab-etats-financiers-annuels-31-12-2017.pdf',
 'adwya-etats-financiers-annuels-31-12-2019.pdf',
 'adwya-etats-financiers-annuels-31-12-2018.pdf',
 'adwya-etats-financiers-annuels-31-12-2016.pdf',
 'aetec-etats-financiers-annuels-31-12-2019.pdf',
 'aetec-etats-financiers-annuels-31-12-2018.pdf',
 'aetec-etats-financiers-annuels-31-12-2017.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'al-etats-financiers-annuels-consolides-31-12-2017.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'alkim-etats-financiers-annuels-31-12-2018.pdf',
 'alkim-etats-financiers-annuels-consolides-31-12-2018.pdf',
 'ams-etats-financiers-annuels-31-12-2017.pdf',
 'ams-etats-financiers-annuels-consolides-31-12-2015.pdf',
 'artes-etats-financiers-annuels-consolides-31-12-2019.pdf',
 'artes-etats-financier